In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
from daq.pico import CSV
from scipy.optimize import fmin
from easier import ParamState
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, Ridge
hv.extension('bokeh')

In [ ]:
def get_freq(df, freq_guess):
    x = df.t.values
    y = df.sig_gen.values

    p = ParamState(
        'x',
        'y_true',
        a=1,
        f=freq_guess,
        phi=0
    )
    p.given(
        x=x,
        y_true=y
    )

    def model(p):
        return p.a * np.sin(2 * np.pi * p.f * x + p.phi)

    def cost(args, p):
        p.ingest(args)
        err = model(p) - p.y_true
        return np.sum(err ** 2)

    x0 = p.array
    xf = fmin(cost, x0, args=(p,), disp=False)
    p.ingest(xf)
    return p.f

In [ ]:
file_name = './data_20171220/20171220-0003.csv'
freq_guess = 20
df = CSV(file_name, a='sig_gen', b='res_volt', c='sec_volt', max_sample_freq=3000000).df
# df = df.head(1000)
# f0 = get_freq(df, freq_guess)
# max_harmonic = 4

# df_basis = pd.DataFrame(index=df.index)

# for n in range(1, max_harmonic + 1):
#     f = n * f0
#     w = 2 * np.pi * f
#     scol = f'sin{n:02d}'
#     ccol = f'cos{n:02d}'
#     df_basis.loc[:, scol] = np.sin(n * w * df.t)
#     df_basis.loc[:, ccol] = np.sin(n * w * df.t)
# df.head()

In [ ]:
def get_basis_frame(df, fundamental_freq, num_freqs):
    f0 = get_freq(df, fundamental_freq)
    df_basis = pd.DataFrame(index=df.index)

    for n in range(1, num_freqs + 1):
        f = n * f0
        w = 2 * np.pi * f
        scol = f'sin{n:02d}'
        ccol = f'cos{n:02d}'
        df_basis.loc[:, scol] = np.sin(n * w * df.t)
        df_basis.loc[:, ccol] = np.cos(n * w * df.t)
    return df_basis   
    


In [ ]:
def run_fit(df, fundamental_freq, num_freqs):
    df_basis = get_basis_frame(df, fundamental_freq, num_freqs)
    
    y_res = df.res_volt
    y_sec = df.sec_volt

#     model = LinearRegression()
    # model = RidgeCV(store_cv_values=True)
    # model = LassoCV()
    alpha = .000002
    
    model_res = Ridge(alpha=alpha)
    model_res.fit(df_basis.values, y_res.values)
    print(model_res.coef_)
    
    model_sec = Ridge(alpha=alpha)
    model_sec.fit(df_basis.values, y_sec.values)
    print(model_sec.coef_)
    
    y_res_fit = model_res.predict(df_basis.values)
    y_sec_fit = model_sec.predict(df_basis.values)
    return y_res_fit, y_sec_fit

file_name = './data_20171220/20171220-0003.csv'
fundamental_freq = 20
num_freqs = 5
y_res_fit, y_sec_fit = run_fit(df, fundamental_freq, num_freqs)

In [ ]:
xxx = df.res_volt - y_res_fit
hv.Histogram(np.histogram(xxx, bins=30))

In [ ]:
%opts Curve [width=500, height=350]
(
   hv.Curve((df.t, df.res_volt)) * hv.Curve((df.t, y_res_fit))
   + hv.Curve((df.t, df.sec_volt)) * hv.Curve((df.t, y_sec_fit))

).cols(1)

In [ ]:
%%opts Scatter [width=600, height=350 logy=True] (size=10)
hv.Scatter((range(len(model.coef_)), np.abs(model.coef_)))

In [ ]:
%opts Curve [width=500, height=350]
(
    hv.Curve(df, kdims=['t'], vdims=['sig_gen'], label='_sig_gen')
    * hv.Curve((df.t, df_basis.sin01))
#     * hv.Curve(df, kdims=['t'], vdims=['sec_volt'], label='_sec_volt')
#     * hv.Curve(df, kdims=['t'], vdims=['res_volt'], label='_res_volt')
)


In [ ]:
get_freq(df, 20)